In [1]:
from IPython.display import clear_output
import time
import os
import datetime

from tools import *

play_beep()

from IPython.display import display
from IPython.core.display import HTML


def display_full_output():
    display(HTML("<style>.output_subarea { overflow: auto; }</style>"))


display_full_output()

In [2]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=10)
# util.logToConsole()

<IB connected to 127.0.0.1:4001 clientId=10>

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 317, reqId 46: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfuture; usfarm; secdefil. The following farms are not connected: ushmds.
Error 317, reqId 46: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has

In [3]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

In [4]:
ib.cancelMktDepth(contract)

In [4]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685),
 Position(account='U10394496', contract=Option(conId=682674010, symbol='AAPL', lastTradeDateOrContractMonth='20241018', strike=150.0, right='P', multiplier='100', currency='USD', localSymbol='AAPL  241018P00150000', tradingClass='AAPL'), position=2.0, avgCost=244.69685),
 Position(account='U10394496', contract=Option(conId=669333316, symbol='NVDA', lastTradeDateOrContractMonth='20240719', strike=555.0, right='P', multiplier='100', currency='USD', localSymbol='NVDA  240719P00555000', tradingClass='NVDA'), position=1.0, avgCost=493.0494),
 Position(account='U10394496', contract=Option(conId=613471094, symbol='GOOGL', lastTradeDateOrContractMonth='20240920', strike=120.0, right='P', multiplier='100', currency='USD', localSym

In [4]:
# default reset values
buy2open = None
# get a specific order by id
trades = get_all_openorders(ib)
sell2close = [trade for trade in trades if trade.order.permId == 1939815533][0]
print_order(sell2close)

future = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]

tick_tp = 10
ticrement = 0.25

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	


In [5]:
########################################
## BUY TO OPEN SCALP 0.1
########################################

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if buy2open is None and sell2close is None:
        os.system("afplay /System/Library/Sounds/Submarine.aiff")
        print("Placing L1.bto")
        action = "BUY"
        totalQuantity = 1
        lmtPrice = ticker.domBids[0].price - tick_tp * ticrement
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        print()

        limit_bto = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )
        buy2open = ib.placeOrder(contract, limit_bto)
        buy2open_ts = datetime.datetime.now()
        ib.sleep(1)

    print(f"BUY TO OPEN:")
    print_order(buy2open)
    print()

    if buy2open is not None:
        if buy2open.orderStatus.status == "Submitted" and sell2close is None:
            print(
                f"Waiting to get filled L1.bto {buy2open.order.permId} {buy2open.orderStatus.status}"
            )

            if datetime.datetime.now() - buy2open_ts > datetime.timedelta(minutes=5):
                print("Cancelling order due to timeout")
                ib.cancelOrder(buy2open.order)
                ib.sleep(1)
                print(buy2open.log)
            print()
        if buy2open.orderStatus.status == "Filled" and sell2close is None:
            os.system("afplay /System/Library/Sounds/Submarine.aiff")
            print("Placing L2.stc")
            action = "SELL"
            totalQuantity = 1
            lmtPrice = buy2open.orderStatus.avgFillPrice + tick_tp * ticrement
            print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
            limit_stc = LimitOrder(
                action=action,
                totalQuantity=totalQuantity,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
            sell2close = ib.placeOrder(contract, limit_stc)
            ib.sleep(1)
        elif (
            buy2open.orderStatus.status == "Inactive"
            or buy2open.orderStatus.status == "Cancelled"
        ) and sell2close is None:
            print("***** order is inactive *****")
            print(buy2open.log)
            print("*****************************")
            buy2open = None

    print(f"SELL TO CLOSE:")
    print_order(sell2close)

    if sell2close is not None:
        if sell2close.orderStatus.status == "Filled":
            os.system("afplay /System/Library/Sounds/Submarine.aiff")
            print("L2.stc filled @ {}".format(sell2close.orderStatus.avgFillPrice))
            buy2open = None
            sell2close = None
    print()
    print()
    print("ALL OPEN ORDERS:")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract NQM2024
    print("Current position:")
    print(
        f"{future.contract.symbol}:  {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )

    print_account_summary(ib=ib)
    print()
    for i in range(5):
        print(".", end="")
        time.sleep(1)

BUY TO OPEN:
None

SELL TO CLOSE:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	


ALL OPEN ORDERS:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	
NQ	1056387955	PreSubmitted	SELL	0.0	2.0		20480.0	

Current position:
NQ:  -6.0 @ 17905.0535435925
AccountType : INDIVIDUAL
Cushion : 0.265488
LookAheadNextChange : 1714483800
AccruedCash : 0.00
AvailableFunds : 37124.81
BuyingPower : 203954.02
EquityWithLoanValue : 50988.51
ExcessLiquidity : 50988.51
FullAvailableFunds : 37124.81
FullExcessLiquidity : 50988.51
FullInitMarginReq : 152500.69
FullMaintMarginReq : 138636.99
GrossPositionValue : 2430.40
InitMarginReq : 152500.69
LookAheadAvailableFunds : 49644.45
LookAheadExcessLiquidity : 92579.60
LookAh

KeyboardInterrupt: 